In [1]:

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
# from genetic_algorithm import GeneticAlgorithm  # Import the genetic algorithm library
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [2]:
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import losses
import matplotlib.pyplot as plt

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/valarinemichael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/valarinemichael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [5]:
df = pd.read_csv('/Users/valarinemichael/Desktop/SEM -2 /Thesis/Twitter_Data.csv')

In [6]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [7]:
df.dropna(subset=['category'], inplace=True)

In [8]:
stopwords_set = set(stopwords.words('english'))
def preprocess_text(text):
        # Check if text is a string
        if isinstance(text, str):
            # Convert text to lowercase
            text = text.lower()

            # Remove URLs
            text = re.sub(r'http\S+|www\S+', '', text)

            # Remove numbers
            text = re.sub(r'\d+', '', text)

            # Remove punctuation
            text = text.translate(str.maketrans('', '', string.punctuation))

            # Tokenization
            tokens = text.split()

            # Remove stopwords

            tokens = [word for word in tokens if word not in stopwords_set]

            # Lemmatization
            lemmatizer = WordNetLemmatizer()
            tokens = [lemmatizer.lemmatize(word) for word in tokens]

            # Join tokens back into a single string
            cleaned_text = ' '.join(tokens)

            return cleaned_text

        # Return empty string if text is not a string
        return ''

In [9]:
df['clean_text'] = df['clean_text'].apply(preprocess_text)

In [10]:
df.dropna(subset=['clean_text'], inplace=True)
df.dropna(subset=['category'], inplace=True)

In [11]:
X = df['clean_text'].values
# y = tf.keras.utils.to_categorical(df['category'], num_classes=len(df['category'].unique()))

y= df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
y[:10]

0   -1.0
1    0.0
2    1.0
3    1.0
4    1.0
5    0.0
6    0.0
7    0.0
8    1.0
9    1.0
Name: category, dtype: float64

In [13]:
df['category'].unique()

array([-1.,  0.,  1.])

In [14]:
lens = df['clean_text'].str.len()

In [15]:
len([x for x in lens if x> 180])

11504

In [16]:
vocabulary_size = 200000
max_text_len = 180

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(df['clean_text'].values)
le = len(tokenizer.word_index) + 1
print(le)
sequences = tokenizer.texts_to_sequences(df['clean_text'].values)
X_DeepLearning = tf.keras.utils.pad_sequences(sequences, maxlen=max_text_len)

102294


In [17]:
labels = tf.keras.utils.to_categorical(df['category'], num_classes=3)
XX_train, XX_test, y_train, y_test = train_test_split(X_DeepLearning , labels, test_size=0.25, random_state=42)
print((XX_train.shape, y_train.shape, XX_test.shape, y_test.shape))

((122229, 180), (122229, 3), (40744, 180), (40744, 3))


In [18]:
from tensorflow.keras.layers import Activation, Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten, GRU, Conv1D, MaxPooling1D, Bidirectional
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [19]:
X_DeepLearning.shape

(162973, 180)

In [20]:
epochs = 10
emb_dim = 256
batch_size = 50
model_lstm1 = Sequential()
model_lstm1.add(tf.keras.Input(shape=(X_DeepLearning.shape[1],)))
model_lstm1.add(Embedding(vocabulary_size,emb_dim, input_length=X_DeepLearning.shape[1],) )
model_lstm1.add(SpatialDropout1D(0.8))
model_lstm1.add(Bidirectional(LSTM(300, dropout=0.5, recurrent_dropout=0.5)))
model_lstm1.add(Dropout(0.5))
model_lstm1.add(Flatten())
model_lstm1.add(Dense(64, activation='relu'))
model_lstm1.add(Dropout(0.5))
model_lstm1.add(Dense(3, activation='softmax'))
model_lstm1.compile(optimizer=tf.optimizers.Adam(),loss='categorical_crossentropy', metrics=['acc'])
print(model_lstm1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 180, 256)          51200000  
                                                                 
 spatial_dropout1d (Spatial  (None, 180, 256)          0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 600)               1336800   
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 600)               0         
                                                                 
 flatten (Flatten)           (None, 600)               0         
                                                                 
 dense (Dense)               (None, 64)                3

In [21]:


checkpoint_callback = ModelCheckpoint(filepath="lastm-1-layer-best_model.h5", save_best_only=True, monitor="val_acc", mode="max", verbose=1)

early_stopping_callback = EarlyStopping(monitor="val_acc", mode="max", patience=10, verbose=1, restore_best_weights=True)

reduce_lr_callback = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, verbose=1, mode="min", min_delta=0.0001, cooldown=0, min_lr=0)

callbacks=[checkpoint_callback, early_stopping_callback, reduce_lr_callback]



In [22]:
history_lstm1 = model_lstm1.fit(XX_train, y_train, epochs = epochs, batch_size = 250, validation_data=(XX_test, y_test), callbacks=callbacks)

Epoch 1/10
489/489 [==============================] - ETA: 0s - loss: 0.7182 - acc: 0.6985
Epoch 1: val_acc improved from -inf to 0.86501, saving model to lastm-1-layer-best_model.h5
489/489 [==============================] - 985s 2s/step - loss: 0.7182 - acc: 0.6985 - val_loss: 0.4002 - val_acc: 0.8650 - lr: 0.0010
Epoch 2/10
489/489 [==============================] - ETA: 0s - loss: 0.4068 - acc: 0.8683
Epoch 2: val_acc improved from 0.86501 to 0.89608, saving model to lastm-1-layer-best_model.h5
489/489 [==============================] - 1025s 2s/step - loss: 0.4068 - acc: 0.8683 - val_loss: 0.3374 - val_acc: 0.8961 - lr: 0.0010
Epoch 3/10
489/489 [==============================] - ETA: 0s - loss: 0.3418 - acc: 0.8938
Epoch 3: val_acc improved from 0.89608 to 0.90340, saving model to lastm-1-layer-best_model.h5
489/489 [==============================] - 1014s 2s/step - loss: 0.3418 - acc: 0.8938 - val_loss: 0.3157 - val_acc: 0.9034 - lr: 0.0010
Epoch 4/10
489/489 [==================

In [23]:
import tensorflow as tf

# Assuming XX_train and y_train are your feature and label arrays
# Create a tuple of feature and label arrays
data_tuple = (XX_train, y_train)

# Create a TensorFlow Dataset
train_ds = tf.data.Dataset.from_tensor_slices(data_tuple)

In [24]:
len(train_ds)

122229

In [25]:
test_ds = tf.data.Dataset.from_tensor_slices((XX_test, y_test))

In [26]:
list(train_ds.take(1))[0][1]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 1.], dtype=float32)>

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [29]:
X = df['clean_text'].values
# y = tf.keras.utils.to_categorical(df['category'], num_classes=len(df['category'].unique()))

y= df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [31]:
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train_tfidf, y_train)

LogisticRegression()

In [32]:
y_train

66956     1.0
110057    1.0
64039     1.0
79604    -1.0
59915     1.0
         ... 
119879   -1.0
103694    0.0
131933   -1.0
146868    1.0
121958    0.0
Name: category, Length: 130378, dtype: float64

In [33]:
_pred = logistic_regression.predict(X_test_tfidf)

In [34]:
_pred[:10]

array([ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1., -1.])

In [35]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [36]:
accuracy_score(y_test, _pred)

0.879306642123025

In [37]:
import numpy as np

In [38]:
y_train.to_list()

[1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 -1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 -1.0,
 0.0,
 -1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 1.0,
 -1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 -1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 -1.0,
 -1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 -1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 1.0,
 -1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 -1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 -1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 -1.0,
 0.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 0.0,
 1.0,
 -1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 -1.0,
 1.0,
 0.0

In [39]:
from sklearn.svm import SVC

In [40]:
np.any(np.isnan(y_train))

False

In [41]:
svm = SVC(decision_function_shape='ovo')
svm.fit(X_train_tfidf, y_train.to_list())

SVC(decision_function_shape='ovo')

In [42]:
svc_prd = svm.predict(X_test_tfidf)

In [43]:
accuracy_score(y_test, svc_prd)

0.8833563429973922

In [44]:
import numpy as np

class GeneticAlgorithm:
    def __init__(self, population_size, num_features, num_generations):
        self.population_size = population_size
        self.num_features = num_features
        self.num_generations = num_generations
        self.population = None
        self.fitness_scores = None
        self.selected_features = None

    def initialize_population(self):
        self.population = np.random.randint(2, size=(self.population_size, self.num_features))

    def calculate_fitness(self, model, X_train, X_test, y_train, y_test):
        self.fitness_scores = []
        for individual in self.population:
            selected_features = [i for i in range(self.num_features) if individual[i] == 1]
            X_train_selected = X_train[:, selected_features]
            X_test_selected = X_test[:, selected_features]
            model.fit(X_train_selected, y_train)
            y_pred = model.predict(X_test_selected)
            accuracy = accuracy_score(y_test, y_pred)
            self.fitness_scores.append(accuracy)

    def selection(self):
        tournament_size = 3
        selected_indices = []
        for _ in range(self.population_size):
            tournament_indices = np.random.choice(range(self.population_size), size=tournament_size, replace=False)
            tournament_fitness = [self.fitness_scores[i] for i in tournament_indices]
            winner_index = tournament_indices[np.argmax(tournament_fitness)]
            selected_indices.append(winner_index)
        self.population = self.population[selected_indices]

    def crossover(self):
        offspring = []
        for _ in range(self.population_size):
            parent_indices = np.random.choice(range(self.population_size), size=2, replace=False)
            parent1, parent2 = self.population[parent_indices]
            parent1 = parent1.ravel()  # Reshape to 1D array
            parent2 = parent2.ravel()  # Reshape to 1D array
            crossover_point = np.random.randint(1, self.num_features)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            offspring.append(child)
        self.population = np.array(offspring)

    def mutation(self, mutation_rate):
        for individual in self.population:
            for i in range(self.num_features):
                if np.random.rand() < mutation_rate:
                    individual[i] = 1 - individual[i]

    def optimize(self, model, X_train, X_test, y_train, y_test):
        self.initialize_population()
        for _ in range(self.num_generations):
            self.calculate_fitness(model, X_train, X_test, y_train, y_test)
            self.selection()
            self.crossover()
            self.mutation(mutation_rate=0.01)
        best_individual_index = np.argmax(self.fitness_scores)
        self.selected_features = [i for i in range(self.num_features) if self.population[best_individual_index][i] == 1]
        return self.selected_features

In [45]:
# Create an instance of the GeneticAlgorithm class
ga = GeneticAlgorithm(population_size=10, num_features=X_train_tfidf.shape[1], num_generations=5)

In [ ]:
# Run the genetic algorithm to optimize feature selection
selected_features = ga.optimize(logistic_regression, X_train_tfidf, X_test_tfidf, y_train, y_test)

In [ ]:
## Apply feature selection to the training and testing data
X_train_selected = X_train_tfidf[:, selected_features]
X_test_selected = X_test_tfidf[:, selected_features]

In [ ]:
#- List of models and their names
models = [svm_model, logistic_regression_model]
model_names = ['SVM', 'Logistic Regression']

In [ ]:
# Initialize a figure for ROC curve plotting
plt.figure(figsize=(10, 8))

In [ ]:
# Loop through each model and print evaluation results
for i, model in enumerate(models):
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    classification_result = classification_report(y_test, y_pred)  # Change the variable name here
    confusion_matrix_result = confusion_matrix(y_test, y_pred)  # Change the variable name here
    
    # ROC curve calculation
    y_pred_prob = model.predict_proba(X_test_selected)
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    
    # Print evaluation results
    print(f"{model_names[i]} Model:")
    print("Accuracy:", accuracy)
    print("Classification Report:\n", classification_result)
    
    # Plot ROC curve
    plt.plot(fpr, tpr, label=f'{model_names[i]} (AUC = {roc_auc:.2f})')

In [ ]:
# Add labels and legend to the plot
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# List of models and their names
models = [svm]
model_names = ['SVM']

In [ ]:
# List to store accuracy scores
accuracy_scores = []

In [ ]:
# Evaluate each model and store the accuracy score
for model in models:
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

In [ ]:
from matplotlib import pyplot as plt